In [1]:
import sys
sys.path.append('../')
from pprint import pprint
import time
import pickle
import importlib
import numpy as np
import time

In [2]:
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter
from libs_qrem import IgnisFilter

/var/folders/vr/d82vcp297v78vk4sb0_nzzdc0000gn/T/ipykernel_17658/4108104578.py:1: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import TensoredMeasFitter


In [3]:
max_iter = 10
min_size = 1
max_size = 30

In [4]:
with open("pkls/raw_hist_list_list.pkl", "rb") as f:
    raw_hist_list_list = pickle.load(f)
with open("pkls/cal_matrices_list_list.pkl", "rb") as f:
    cal_matrices_list_list = pickle.load(f)

In [5]:
ignis_mitigator_list_list = []
for num_qubits in range(min_size, 25 + 1):
    t1 = time.perf_counter()
    ignis_mitigator_list = []
    for i in range(0, max_iter):
        ignis_mitigator_list.append(IgnisFilter(num_qubits, cal_matrices_list_list[num_qubits - 1][i]))
        if i % 10 == 0:
            print("iter", i, "finished")
    ignis_mitigator_list_list.append(ignis_mitigator_list)
    t2 = time.perf_counter()
    print("length of ignis_mitigator_list: ", len(ignis_mitigator_list))
    print("size", num_qubits, "finished (t=", t2 - t1, "s)")
    print()

iter 0 finished
length of ignis_mitigator_list:  10
size 1 finished (t= 0.00047669200000033385 s)

iter 0 finished
length of ignis_mitigator_list:  10
size 2 finished (t= 0.00027789000000044695 s)

iter 0 finished
length of ignis_mitigator_list:  10
size 3 finished (t= 0.00040887399999967045 s)

iter 0 finished
length of ignis_mitigator_list:  10
size 4 finished (t= 0.00044639499999998833 s)

iter 0 finished
length of ignis_mitigator_list:  10
size 5 finished (t= 0.0005739259999995028 s)

iter 0 finished
length of ignis_mitigator_list:  10
size 6 finished (t= 0.0006911590000004963 s)

iter 0 finished
length of ignis_mitigator_list:  10
size 7 finished (t= 0.0008661979999997627 s)

iter 0 finished
length of ignis_mitigator_list:  10
size 8 finished (t= 0.0008325049999999834 s)

iter 0 finished
length of ignis_mitigator_list:  10
size 9 finished (t= 0.0008545800000003823 s)

iter 0 finished
length of ignis_mitigator_list:  10
size 10 finished (t= 0.0009603629999999086 s)

iter 0 finished

In [6]:
def get_info(mitigator):
    return {"exact_one_norm_of_reduced_inv_A": mitigator.exact_one_norm_of_reduced_inv_A(),
            "mitigated_hist": mitigator.mitigated_hist(),
            "times": mitigator.times(),
            "mitigation_stddev": mitigator.mitigation_stddev(norm_type = "exact"),
            }

In [7]:
ignis_info_list_list = []
ignis_hist_list_list = []
for i in range(min_size - 1, 25):
    t1 = time.perf_counter()
    ignis_info_list = []
    ignis_hist_list = []
    for j in range(0, max_iter):
        ignis_hist_list.append( ignis_mitigator_list_list[i][j].apply(raw_hist_list_list[i][j]) )
        ignis_info_list.append( get_info(ignis_mitigator_list_list[i][j]) )
        print("iter", j, "finished")
    ignis_hist_list_list.append( ignis_hist_list )
    ignis_info_list_list.append( ignis_info_list )
    t2 = time.perf_counter()
    print("size", i + 1, "finished (t=", t2 - t1, "s)")
    print()

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
size 1 finished (t= 0.0014828550000007823 s)

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
size 2 finished (t= 0.002188505999999535 s)

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
size 3 finished (t= 0.0014226080000003805 s)

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
size 4 finished (t= 0.0008541209999997079 s)

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
size 5 finished (

In [8]:
with open("pkls/ignis_info_list_list.pkl", "wb") as f:
    pickle.dump(ignis_info_list_list, f)
with open("pkls/ignis_hist_list_list.pkl", "wb") as f:
    pickle.dump(ignis_hist_list_list, f)